## s000 environment setup

### s001 libraries

In [0]:
import pyspark
import os
from pyspark import sql 
from pyspark.sql import Window
from pyspark.sql import functions as f

# ------------- Use snowflake utility
sfUtils = sc._jvm.net.snowflake.spark.snowflake.Utils

In [0]:
# ------------ login to snowflake
password = dbutils.secrets.get(scope = "auea-kv-sbx-dxdtlprdct01", key = "sfdbrsdskey")

options = {
  "sfUrl": "vodafonenz_prod.australia-east.azure.snowflakecomputing.com/", 
  "sfUser": "SVC_LAB_DS_DATABRICKS",
  "pem_private_key": password.replace('\\n', '\n'),
  "sfDatabase": "LAB_ML_STORE",
  "sfSchema": "RAW",
  "sfWarehouse": "LAB_DS_WH_SCALE"
}

In [0]:
df_aod_hist = (
    spark
    .read
    .format("snowflake")
    .options(**options)
    .option(
        "query"
        , "select * from LAB_ML_STORE.SANDBOX.SC_ADHOC_AOD_HIST_12M"
    )
    .load()
)

In [0]:
df_prm_ifp_bill = spark.read.format('delta').load('/mnt/feature-store-prod-lab/d300_primary/d301_mobile_oa_consumer/prm_ifp_main_on_bill')
df_prm_ifp_srvc = spark.read.format('delta').load('dbfs:/mnt/feature-store-prod-lab/d300_primary/d301_mobile_oa_consumer/prm_ifp_main_on_service')

df_wo_mvnt = spark.read.format('delta').load('/mnt/feature-store-prod-lab/d500_movement/d501_mobile_oa_consumer/mvmt_writeoff')

In [0]:
display(df_prm_ifp_bill.limit(10))

In [0]:
display(df_prm_ifp_srvc.limit(10))

In [0]:
df_ifp_bill = (
    df_prm_ifp_bill
    .filter(f.col('ifp_type') == 'device')
    .filter(f.col('ifp_term_start_date').isNotNull())
    .select('fs_cust_id', 'fs_acct_id', 'fs_ifp_id', 'ifp_term_start_date', 'ifp_term_end_date', 'ifp_sales_channel', 'ifp_sales_channel_branch', 'ifp_sales_channel_group')
    .distinct()
)

In [0]:
df_ifp_srvc=(
    df_prm_ifp_srvc
    .filter(f.col('ifp_type') == 'device')
    .filter(f.col('ifp_term_start_date').isNotNull())
    .select('fs_cust_id', 'fs_acct_id', 'fs_ifp_id', 'ifp_term_start_date', 'ifp_term_end_date', 'ifp_sales_channel', 'ifp_sales_channel_branch', 'ifp_sales_channel_group')
    .distinct()
)

In [0]:
df=(
    df_ifp_bill
    .union(df_ifp_srvc)
)

In [0]:
window_spec = Window.partitionBy("fs_cust_id").orderBy("ifp_term_start_date")

df_with_sequence = df.withColumn("ifp_sequence", f.row_number().over(window_spec))

# Add previous IFP end date for each customer's IFP
df_with_prev = (
    df_with_sequence
    .withColumn(
    "prev_ifp_end_date", 
    f.lag("ifp_term_end_date", 1).over(Window.partitionBy("fs_cust_id").orderBy("ifp_term_start_date"))
    )
    .withColumn(
    "prev_ifp_start_date", 
    f.lag("ifp_term_start_date", 1).over(Window.partitionBy("fs_cust_id").orderBy("ifp_term_start_date"))
    )
)

In [0]:
df_with_rollup = (
    df_with_prev
    .withColumn(
    "is_rollup" 
    , f.when(
        (f.col("prev_ifp_end_date").isNotNull()) & 
        (f.col("ifp_term_start_date") < f.col("prev_ifp_end_date"))
        , True
    ).otherwise(False)
)
)

In [0]:
display(
    df_with_rollup
    .select('ifp_sales_channel_branch', 'ifp_sales_channel_group','ifp_sales_channel')
    .distinct()    
)

In [0]:
display(df_aod_hist.limit(10))

df_aod_latest = (
    df_aod_hist
    .filter(f.col('snapshot_month') == '2025-03-31')
)

# aod latest 
display(
    df_aod_latest
    .limit(10)
)

In [0]:
display(
    df_with_rollup
    .withColumn('ifp_start_month_end', f.last_day('ifp_term_start_date'))
    .filter(f.col('ifp_start_month_end') <'2024-12-31')
    .agg(
        f.count('*')
        , f.countDistinct('fs_ifp_id')
        , f.countDistinct('fs_cust_id')
    )
)

In [0]:
display(
    df_with_rollup
    .withColumn('ifp_start_month_end', f.last_day('ifp_term_start_date'))
    .filter(f.col('ifp_start_month_end')>='2024-12-31')
    .join(df_aod_hist, f.col('account_no') == f.col('fs_acct_id'), 'left')
    .limit(10)
    #.filter(f.col('PAYMENT_STAUS') ==5)
)

In [0]:
#df_aod_hist

display(
    df_with_rollup
    .withColumn('ifp_start_month_end', f.last_day('ifp_term_start_date'))
    .withColumn('three_months_after_start', f.last_day(f.add_months('ifp_start_month_end', 3)))
    .filter(f.col('ifp_start_month_end')>='2024-01-01')
    .filter(f.col('three_months_after_start') <= '2025-03-31')
    .join(df_aod_hist
          ,  (f.col('account_no') == f.col('fs_acct_id')) & 
             (f.col('three_months_after_start') == f.col('SNAPSHOT_MONTH'))
          , 'left')
    .groupBy('PAYMENT_STAUS', 'ifp_start_month_end', 'SNAPSHOT_MONTH',  'is_rollup', 'ifp_sales_channel')
    .agg(
        f.countDistinct('fs_ifp_id')
        , f.countDistinct('fs_cust_id')
    )
)




In [0]:
df_wo_base = (
    df_wo_mvnt
    .select('fs_cust_id', 'fs_acct_id', 'movement_date', 'movement_type', 'writeoff_item_no')
    .distinct()
)

In [0]:
#df_wo_hist

display(
    df_with_rollup
    .withColumn('ifp_start_month_end', f.last_day('ifp_term_start_date'))
    .filter(f.col('ifp_start_month_end') < '2024-12-31')
    .filter(f.col('ifp_start_month_end') >= '2024-01-01')
    .join(df_wo_base, ['fs_acct_id', 'fs_cust_id'], 'left')
    .withColumn(
        'valid_writeoff_4m' 
        , f.when(
            (f.col('movement_type') == 'writeoff') & 
            (f.col('movement_date').isNotNull()) & 
            (f.col('movement_date') > f.col('ifp_term_start_date')) &
            (f.col('movement_date') <= f.last_day(f.add_months(f.col('ifp_term_start_date'), 4)))
            , 1
        ).otherwise(0)
    )
    .groupBy('ifp_start_month_end', 'ifp_sales_channel', 'is_rollup') 
    .agg(
        # Count total IFPs
        f.count('fs_ifp_id').alias('total_ifps'),
        f.count('fs_cust_id').alias('total_customer'),
        # Count IFPs with valid write-offs
        f.sum('valid_writeoff_4m').alias('writeoff_count')
    )
)
